In [61]:
%matplotlib inline
import sys
import os
sys.path.append('/home/kihyun/data/projects/kihyun/Editing-benchmark/cocoapi/PythonAPI')
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import pylab
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [50]:
dataDir='/home/kihyun/data/dataset/COCO2017'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
json_dir = '/home/eugene/data/projects/eugene/Editing-benchmark/object.json'
Annotation = True # Annotation mask를 이미지에 표시할지 여부
save_dir = './editval_json_test/{}'.format(dataType) # 프롬프트 저장 위치
# save_img = True
img_num = 30

In [43]:
# initialize COCO api for instance annotations
# annotation file을 읽어서 COCO object를 생성
coco_all=COCO(annFile)

loading annotations into memory...
Done (t=18.12s)
creating index...
index created!


In [44]:
with open(json_dir, 'r') as file:
    coco_find=json.load(file)

In [47]:
catnms = list(coco_find.keys())
catnms

['bench',
 'pizza',
 'cup',
 'dog',
 'stop_sign',
 'sink',
 'person',
 'backpack',
 'handbag',
 'airplane',
 'boat',
 'bicycle',
 'couch',
 'chair',
 'donut',
 'car',
 'apple',
 'cat',
 'potted_plant']

In [58]:
def img_show(coco, imgId, mask=False, catnms=None, save_dir=save_dir):
    img = coco.loadImgs(imgId)[0]
    I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
    #I = io.imread(img['coco_url'])
    plt.axis('off')
    plt.imshow(I)
    if mask:
        catIds = coco.getCatIds(catNms=catnms)
        annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        coco.showAnns(anns)
    # save image
    save_name = f'{imgId}.jpg'
    img_path = os.path.join(save_dir, save_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    plt.savefig(img_path)
    plt.clf()
    
def multi_img_show(coco, imgId_ls, mask=False, catnms=None, save_dir=save_dir):
    for imgId in imgId_ls:
        img_show(coco, imgId, mask, catnms, save_dir=save_dir)
        
def save_edit_prompts(catnm, imageId, json_file, save_dir):
    # save edit prompts in prompt_save_dir
    file_name = f'edit_prompts_{catnm}_{imageId}.txt'
    txt_path = os.path.join(save_dir, file_name)
    ff = open(txt_path, 'w')
    for edit_task in json_file[catnm][imageId].keys():
        ff.write(f'{edit_task}\n')
        for subtask1 in json_file[catnm][imageId][edit_task]["from"]:
            if subtask1=="":
                for subtask2 in json_file[catnm][imageId][edit_task]["to"]:
                    ff.write(f'from {catnm} to {subtask2}\n')
            else:
                for subtask2 in json_file[catnm][imageId][edit_task]["to"]:
                    ff.write(f'from {subtask1} to {subtask2}\n')
        ff.write('---------------------\n')
    ff.close()

In [59]:
# preposition 종류
position_set = set()

for class_name in catnms:
    for imageId in coco_find[class_name].keys():
        for edit_task in coco_find[class_name][imageId].keys():
            positions = coco_find[class_name][imageId][edit_task]
            for p in positions:
                position_set.add(p)
print(position_set)

{'to', 'from'}


In [60]:
### image, prompt 따로 저장하는 코드

for class_name in catnms:
    imageIds = coco_find[class_name].keys()
    class_dir = os.path.join(save_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    multi_img_show(coco_all, list(map(int, imageIds)), class_name, save_dir=class_dir)
    for imageId in imageIds:
        save_edit_prompts(class_name, imageId, coco_find, class_dir)

<Figure size 800x1000 with 0 Axes>

In [75]:
### image, prompt를 한 이미지에 같이 저장하는 코드

def img_with_prompt(coco, imgId, json_file, catnms, save_dir):
    img = coco.loadImgs(int(imgId))[0]
    # img_path = os.path.join('images', img['file_name'])  # 실제 이미지 경로를 맞게 수정하세요.
    # I = io.imread(img_path)
    I = io.imread('%s/images/%s/%s'%(dataDir,dataType,img['file_name']))
    
    # Prepare text for prompts
    prompt_text = ""
    for edit_task in json_file[catnms][imgId].keys():
        prompt_text += f'{edit_task}\n'
        for subtask1 in json_file[catnms][imgId][edit_task]["from"]:
            if subtask1 == "":
                for subtask2 in json_file[catnms][imgId][edit_task]["to"]:
                    prompt_text += f'from {catnms} to {subtask2}\n'
            else:
                for subtask2 in json_file[catnms][imgId][edit_task]["to"]:
                    prompt_text += f'from {subtask1} to {subtask2}\n'
        prompt_text += '---------------------\n'

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(10, 10))

    # Display the image
    ax.imshow(I)
    ax.axis('off')  # Hide axis
    
    # Add prompt text below the image
    plt.figtext(1.0, 0.9, prompt_text, ha='left', va='top', fontsize=10, bbox={"facecolor": "white", "alpha": 0.8, "pad": 10})

    # Save the combined image and prompts
    save_name = f'{imgId}_with_prompt.jpg'
    img_save_path = os.path.join(save_dir, save_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    plt.savefig(img_save_path, bbox_inches='tight', pad_inches=0.2)
    plt.close()

def multi_img_with_prompt(coco, imgId_ls, json_file, catnms, save_dir):
    for imgId in imgId_ls:
        img_with_prompt(coco, imgId, json_file, catnms, save_dir)

# Example usage
# coco_find = json.load(open('path_to_your_json_file.json', 'r'))  # Load JSON file
# catnms = ['bench']  # Example category names
# save_dir = 'path_to_save_directory'

for class_name in catnms:
    imageIds = list(coco_find[class_name].keys())
    class_dir = os.path.join(save_dir, class_name)
    multi_img_with_prompt(coco_all, imageIds, coco_find, class_name, save_dir=class_dir)